## Lexis Nexis API for Data Extraction
This notebook accesses the LexisNexis API using the lexisnexisapi Python package. Moreover, it tokenizes the articles into sentences and parses them as CSV

Some useful links for querry development:
- https://solutions.nexis.com/wsapi/news-and-directories/news/
- https://www.lexisnexis.com/pdf/lexis-advance/terms-and-connectors.pdf

### Query to be used below
The query elicitation was explained in the written master thesis

In [1]:
query = ' W/25 (supply OR acquire OR support OR buy OR invest OR procure OR provide OR source OR deliver OR purchase OR obtain OR order OR request)  AND ( ("car" OR ALLCAPS(!EV) OR "e-vehicle") BUT NOT scooter BUT NOT bike) AND (battery OR "storage" OR "lithium ion" OR "cell" OR  "modpack" OR "mod pack" OR)'    

#query for filtering the indexed information of LexisNexis through the API call function
filter_date_eng = "(year(Date) eq 2024 or year(Date) eq 2023 or year(Date) eq 2022 or year(Date) eq 2021 or year(Date) eq 2020 or year(Date) eq 2019) and Language eq LexisNexis.ServicesApi.Language'English'"

### Libraries

In [2]:
import os
import json
import pandas as pd

import bs4 as bs

from nltk.tokenize import sent_tokenize

from lexisnexisapi import webservices as ws
from lexisnexisapi import credentials as cred

### Preparing the NAATBatt list

In [3]:
#reading in the data for the NAATBAtt list - data can be downloaded in excel from the org's website

file_path = 'naatbatt_sc.xlsx'

# sheets in scope of the thesis - ModPack and Electrodes and Cells
sheet_name_1 = '5-ModPack' 
sheet_name_2 = '4-Electrodes and Cells'

# reading in the two sheets
excel_1 = pd.read_excel(file_path, sheet_name=sheet_name_1)
excel_2 = pd.read_excel(file_path, sheet_name=sheet_name_2)

In [4]:
excel_1['file'] = 'ModPack'
excel_2['file'] = 'ElectrodesCells'

excel = pd.concat([excel_1, excel_2], axis = 0)
excel.head()

,ID,Status,Supply Chain Segment,Company,NAATBatt Member,Facility Name,Product Type,Product,Facility or Company Website,Facility Address,...,HQ Company,HQ Website,HQ City,HQ State or Province,HQ Country,QC,QC Date,Sources,Notes,file
0,5000,C,Downstream,3M,No,3M Guin,Non-cell Components,Thermal Systems,3m.com,6675 US Highway 43,...,3M,3m.com,Saint Paul,MN,US,EW,2021-08-09 00:00:00,3m.com,Thermal Pads & Fillers,ModPack
1,5001,C,Downstream,3M,No,3M Cottage Grove,Non-cell Components,Thermal Systems,3m.com,10746 Innovation Road,...,3M,3m.com,Saint Paul,MN,US,EW,2021-08-09 00:00:00,3m.com,Thermal Pads & Fillers,ModPack
2,5002,C,Downstream,3M,No,3M Springfield,Non-cell Components,Safety Systems,3m.com,3211 E Chestnut Expressway,...,3M,3m.com,Saint Paul,MN,US,EW,2021-08-09 00:00:00,3m.com,Rapid thermal suppression,ModPack
3,5003,C,Downstream,AA Portable Power Corp.,No,AA Portable Power Corp.,Cell Assemblies/ Groupings,Packs,aaportablepower.com,825 South 19th Street,...,AA Portable Power Corp.,batteryspace.com,Richmond,CA,US,EW,2021-08-09 00:00:00,Thomasnet: Battery Packs / manufacturers; batt...,"Mostly a battery distributor, but has producti...",ModPack
4,5004,C,Downstream,"ADA Technologies, Inc.",No,"ADA Technologies, Inc.",Non-cell Components,Thermal Systems,adatech.com,11149 Bradford Road,...,"ADA Technolgies, Inc.",adatech.com,Littleton,CO,US,VP,2022-08-01 00:00:00,adatech.com; Questionnaire; https://www.buzzfi...,Thermal Interface Materials;,ModPack


In [5]:
companies_search = excel[['Company', 'Product', 'Product Type', 'file']]

#removing noise from the company names
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Inc",""))
#if we would not enforce ALLCAPS for 3M, hits for other subjects would return - for e.g. 3m as an abbreviation for 3 million 
companies_search.Company = companies_search.Company.apply(lambda x: x.replace("3M", "ALLCAPS(3M)"))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Inc ",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(".",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(",",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Corp",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Company",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" LLC",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Ltd",""))
#stand for Joint Venture of Ford and SK On - the name of the joint venture is enough to use in the query
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" (JV of Ford and SK On)",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Co",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" JV",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" SDI",""))
companies_search.Company = companies_search.Company.apply(lambda x: x.replace("-",'" AND "'))
#removing empty spaces from the beginning and the end of the names
companies_search.Company = companies_search.Company.apply(lambda x: x.rstrip(' '))
companies_search.Company = companies_search.Company.apply(lambda x: x.lstrip(' '))

/tmp/ipykernel_4128484/1645062754.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies_search.Company = companies_search.Company.apply(lambda x: x.replace(" Inc",""))
/tmp/ipykernel_4128484/1645062754.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies_search.Company = companies_search.Company.apply(lambda x: x.replace("3M", "ALLCAPS(3M)"))
/tmp/ipykernel_4128484/1645062754.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [6]:
#this drops rows when one company has more facilities - if it has more facilities, it appears multiple times 
companies_search.drop_duplicates(subset = ['Company', 'file'], keep = 'first', ignore_index = True, inplace = True)

/tmp/ipykernel_4128484/2380239521.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  companies_search.drop_duplicates(subset = ['Company', 'file'], keep = 'first', ignore_index = True, inplace = True)


In [7]:
display(companies_search)

,Company,Product,Product Type,file
0,ALLCAPS(3M),Thermal Systems,Non-cell Components,ModPack
1,AA Portable Power,Packs,Cell Assemblies/ Groupings,ModPack
2,ADA Technologies,Thermal Systems,Non-cell Components,ModPack
3,AKASOL,Packs,Cell Assemblies/ Groupings,ModPack
4,Alion Science and Technology,Modules/ Arrays,Cell Assemblies/ Groupings,ModPack
...,...,...,...,...
175,Ultium Cells,NMC,Prismatic (pouch) cells,ElectrodesCells
176,VinFast,Other/ Unknown,Other/ Unknown cells,ElectrodesCells
177,Volkswagen,NaN,NaN,ElectrodesCells
178,Xalt Energy MI,NMC,Prismatic (pouch) cells,ElectrodesCells


### Accessing the News API
#### Functions to be used for accessing the API and Preprocessing the text

In [9]:
def access_text(data):
    '''
        This function receives an XML structure and extracts from it the title of an article,
        the full text, the document id and the date the news was published. 
        The result is then returned in a dictionary.
    '''
    #accessing the XML using beautiful soup and its html parser
    file = bs.BeautifulSoup(data, "html.parser")

    #the body of the API response is nested in this path
    text = file.content.articledoc.bodytext
    
    #getting the full_text
    #text is split by rows in the article so needs to be appended in one string
    full_text = ''
    for paragraph in text:
        full_text = full_text + paragraph.text
    
    #saving some metadata in case we want to filter later on
    title = file.title.text
    entry_id = file.id.text
    published = file.published.text
    
    #making sure the CSVs don't get broken by bad characters
    full_text = full_text.replace(';', '').replace('\n','')
    title = title.replace(';', '')
    
    res = {
        'title':title,
        'text':full_text,
        'doc_id':entry_id,
        'date':published
    }
    
    return res

In [10]:
def iterate_results (data_from_request):
    '''
        This function iterates through the up to 50 articles received in one batch 
        and returns them as rows in a dataframe. It takes as input the raw request response
        The function extracts the title, text and source name for one article 
        and adds it to a pd.df which is then returned.
    '''

    #creating empty df
    result = pd.DataFrame(columns=['title', 'text', 'source'])

    #getting the dictionary of 50 articles
    texts = data_from_request['value']

    for text in texts:
        try:
            content = text['Document']['Content'] #this is the full text
            
            #the title and the content have to be extracted from the XML format
            res = access_text(content)
            res['source'] = text['Source']['Name']#only the source name can be directly accessed
            
            #converting to a dataframe for returning the result
            res = pd.DataFrame(res, index=[0])

            #concatenating the results during the iteration
            result = pd.concat([result, res], ignore_index=True)

        #error handling for possible issues
        except TypeError as e:
            print(f"TypeError occurred: {e} - Skipping this text.")
        except KeyError as e:
            print(f"KeyError occurred: {e} - Skipping this text.")
        
    return result


In [11]:
def take_sentences (company = str, 
                    number = int, 
                    text = str):
    '''
        This function takes as input a company name,
        a number corresponding to how many sentences should be extracted around the identified
        company and a text to work with. 
        The function tokenizes the text into sentences and performs a search for the
        company name from the NAATBatt list. A number of sentences will be extracted around
        the company name. The final result to be returned is a json dictionary containing a unique
        identifier of the match as key and the extracted sentences as values.
    '''
    #tokenizing the sentences using the nltk tokenizer imported in the first command
    sentences = sent_tokenize(text)
    
    #searching the company name in the lowercase
    #searching through lowercase chars so allcaps don't influence the result
    company_lower = company.lower()
    indices = [i for i, sentence in enumerate(sentences) if company_lower in sentence.lower()]
    
    result = {}
    for index in indices:

        # get the last 3(number) sentences before and the next 3(number) sentences after the sentence with the name
        start = max(0, index - number)
        end = min(len(sentences), index + number + 1)

        # extract the required sentences
        surrounding_sentences = sentences[start:end]
        surrounding_sentences = ' '.join(surrounding_sentences)
        #replacing a weird character that could harm the CSV parsing
        surrounding_sentences = surrounding_sentences.replace('•', ' ')

        #key = index, unique identifeir and value = surrounding_sentences
        result [index] = surrounding_sentences
    
    result = json.dumps(result)

    return result

In [12]:
def save_res (res, company, no_artilces = int):  
    '''
        This function takes as input a company name and the articles returned for this company
        as well as the number of articles retrieved for the company
        It saves the articles in a csv file under a data folder
    '''
    # cleaning the company name so it can be used as a file path
    company = company.replace(' ', '_').replace('"','').replace("AND", "").replace("(","").replace(")","").replace('ALLCAPS',"")
    
    # get the correct path syntax for the system in the current wd
    path = os.path.join(os.getcwd(), 'data/'+company)

    # check if the directory already exists
    if not os.path.exists(path):
        # create the directory if it does not
        os.makedirs(path)

    #create the file name and path
    file_name = company+'_'+str(no_artilces)+'_news.csv'
    address = path + '/' + file_name

    #saving the pandas df to the path with ; as separator
    res.to_csv(path_or_buf = address, sep = ';')
    
    return
                  

In [13]:
def check_batchnews_need (search_query):
    '''
        This function takes as input a search query and sends a dummy request
        to the LexisNexis API to obtain the total number of articles for a company. 
        This is required to evaluate if the BatchNews API or the News API must be used, as indicated 
        by the team at LexisNexis. If there are more than 500 articles to be returned for one query,
        than we must use the BatchNews API since it is more efficient on their side.
        The function returns the endpoint to be used and the number of articles identified
    '''
    #filtering the articles by date
    parameters ={
            '$search':search_query,
            '$filter':filter_date_eng,
            '$top': 0, #we don't need any articles now, just the response header
            '$skip':0 
    }

    #api call using the lexisnexisapi for Python
    data = ws.call_api(access_token = token,
                       endpoint = 'News', 
                       params = parameters)

    #extracting the number of articles
    count = data['@odata.count']

    #checking wich endpoint to use
    if count > 500:
        endpoint = 'BatchNews'
        return endpoint, count
    else: 
        endpoint = 'News'
        return endpoint, count

#### Main Code for Querrying data

In [14]:
#create an access token
#enter Client Id, Client Secret Here - to be obtained from WU library
token = ws.token()

In [15]:
import time

start_company = "Xerion Advanced Battery" #for continuing from one particular company in the NAATBatt list, insert the company name here
#we can start from a specified copmany in case an error appeared and we need to restart the downloading from the previously failed point
start_index  = companies_search[companies_search.Company == start_company].index[0]

#iterating through the NAATBatt list of companies
for company in companies_search.Company[start_index:]:
    #cleaning the company name for logging
    company_clean = company.replace('"','').replace("AND", "").replace("(","").replace(")","").replace('ALLCAPS',"").replace('_','')

    #appending the company name to the search query from the notebook beginning
    search_query = company + query 

    #filtering the date, english texts and returning 50 articles at a time (max number of articles to be returned at once)
    parameters ={
            '$search':search_query,
            '$expand':'Document', #A navigation property name which will be included with the current result set.
            '$filter':filter_date_eng,
            '$top': 50,
            '$skip':0
    }
    
    #checking if we need to use News or BatchNews
    endpoint, count = check_batchnews_need(search_query)
    
    #some primitive logging
    print(company_clean, endpoint, count, '\n', search_query)

    #actually calling the API
    data = ws.call_api(access_token = token,
                       endpoint = endpoint, 
                       params = parameters)
            
    #opening the XML returned
    final_res = iterate_results(data)
    
    #transforming the text into a dictionary with only 2 sentences around the key company
    final_res['sentences'] = final_res.text.apply(lambda x: take_sentences(company_clean, 1, x) )
    
    #checking the pagination of the response
    #if more than 50 articles are possible to extract, a link to the next ones appears
    if '@odata.nextLink' in data.keys():
        skip = data['@odata.nextLink'].split('$skip=')[1][:2]
    else: skip =0

    #looping through the pagination - basically sending repeated requests with a wait of 20s for getting all articles
    #the wait is needed to not breach the throttle limits
    while '@odata.nextLink' in data.keys() and int(skip) <= count:

        #adding the skip to the list of parameters so we don't always get the same articles
        parameters['$skip'] = skip
        
        data = ws.call_api(access_token = token,
                           endpoint = endpoint, 
                           params = parameters)  #Set endpoint='News'

        ##opening the XML returned
        res = iterate_results(data)

        #transforming the text into a dictionary with only 2 sentences around the key company
        res['sentences'] = res.text.apply(lambda x: take_sentences(company_clean, 1, x) )
        
        #concatenating the dataframes for having all articles in a single CSV
        frames = [final_res, res]
        final_res = pd.concat(frames, 
                              axis = 0, 
                              ignore_index = True)
        
        #moving on to the next 50 articles
        skip = int(skip) + 50

        #primitve logging again
        print(skip)

        # Pause the execution for 20 seconds
        time.sleep(20)

    #after all articles are returned, saving them to CSV
    save_res(final_res, company, skip)


Xerion Advanced Battery News 14 
 Xerion Advanced Battery W/25 (supply OR acquire OR support OR buy OR invest OR procure OR provide OR source OR deliver OR purchase OR obtain OR order OR request)  AND ( ("car" OR ALLCAPS(!EV) OR "e-vehicle") BUT NOT scooter BUT NOT bike) AND (battery OR "storage" OR "lithium ion" OR "cell" OR  "modpack" OR "mod pack" OR)
